## Parse DB as DataFrame

In [95]:
import pandas as pd
import mysql.connector as connection

In [136]:
try:
    mydb = connection.connect(
        host="192.168.1.52", 
        port="3307",
        database="webscraper_database",
        user="webscraper", 
        passwd="ackj8636&fkf",
        use_pure=True
    )
    query = "Select * from condos;"
    df = pd.read_sql(query,mydb)
    mydb.close() #close the connection
    
    df.set_index("id", inplace=True)
    print(f"Database found with {len(df)} entries")
    
except Exception as e:
    mydb.close()
    print(str(e))

/home/manuel/.virtualenvs/webscraper/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Database found with 2854 entries


## Personal Filter

In [137]:
df = df[df.room_count>=4]

In [138]:
len(df)

116

## Address to Lat/Lon

In [139]:
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm
tqdm.pandas()

In [140]:
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)

In [141]:
df["location"] = df["address"].progress_apply(geocode)

df["point"] = df["location"].apply(lambda loc: tuple(loc.point) if loc else None)
df[["latitude", "longitude", "altitude"]] = pd.DataFrame(df["point"].tolist(), index=df.index)

df = df.drop(columns=["location", "point", "altitude"])

100%|████████████████████████████████████████████████████████████████████████| 116/116 [00:58<00:00,  1.99it/s]


In [142]:
import geopandas as gpd

In [143]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude)).set_crs("epsg:4326")

In [144]:
len(gdf[gdf.is_empty]), len(gdf)

(8, 116)

In [145]:
gdf[["price", "room_count", "price_per_m2", "geometry"]][~gdf.is_empty].explore()

## Graz Information

In [22]:
import geopandas as gpd

In [31]:
gdf = gpd.read_file("graz.kml")

ValueError: Null layer: ''